# **DATA CLEANING, WRANGLING, AND MANIPULATION**

**(Before Data Visualization)**

## Importing Pandas and creating DataFrames

First I import necessary packages and create a DataFrame for each sheet.

In [1]:
import pandas as pd
import os

In [8]:
pwd = os.getcwd()

df1 = pd.read_excel(pwd + r'\updated_coffee_sales_data.xlsx', sheet_name='transactions')
df2 = pd.read_excel(pwd + r'\updated_coffee_sales_data.xlsx', sheet_name='employees')
df3 = pd.read_excel(pwd + r'\updated_coffee_sales_data.xlsx', sheet_name='coffee')

Then I check to see if there are any columns with mismatched Dtypes

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Transaction ID         2494 non-null   object        
 1   Emp_id                 2494 non-null   int64         
 2   Employee Name          2494 non-null   object        
 3   Date and Time          2494 non-null   datetime64[ns]
 4   Purchased Coffee Type  2494 non-null   object        
 5   Purchase Quantity      2494 non-null   int64         
 6   total_price            2494 non-null   float64       
 7   Payment Method         2494 non-null   object        
 8   Customer Name          2494 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 175.5+ KB


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Emp_id         4 non-null      int64 
 1   Emp_Name       4 non-null      object
 2   Job Title      4 non-null      object
 3   Hourly Salary  4 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 260.0+ bytes


In [16]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Coffee type  5 non-null      object 
 1   price        5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 212.0+ bytes


## Merging tables

Everything looks fine so I can proceed.

In this case, I will join all of the tables together to create one DataFrame.

The first step to achieving this is to rename the columns and make the match across all DataFrames.

In [33]:
df1 = df1.rename(columns=
          {'Transaction ID' : 'trn_id', 'Emp_id' : 'emp_id',
          'Employee Name' : 'emp_name', 'Date and Time' : 'date_time',
          'Purchased Coffee Type' : 'coffee_type', 'Purchase Quantity' : 'purchase_qty',
          'Payment Method' : 'pay_method', 'Customer Name' : 'customer_name'}
     ) 
df2 = df2.rename(columns=
          {'Emp_id' : 'emp_id', 'Emp_Name' : 'emp_name',
          'Job Title' : 'job_title', 'Hourly Salary' : 'hr_salary'}
     )
df3 = df3.rename(columns=
          {'Coffee type' : 'coffee_type'}
     )

Now that everything is named properly, I can merge the three DataFrames with no problems.

In [34]:
df_merged = df1.merge(df2, on='emp_id', how='inner').merge(df3, on='coffee_type', how='inner')
df_merged.head()

,trn_id,emp_id,emp_name_x,date_time,coffee_type,purchase_qty,total_price,pay_method,customer_name,emp_name_y,job_title,hr_salary,price
0,80dbda09-0c0f-4360-87b8-82058756a097,98845,Kevin Swanson,2024-08-01 10:17:03,Latte,4,8.0,Card,Brandon Johnson,Kevin Swanson,Jr. Barista,15,2.0
1,a0ee1475-c554-45bc-862e-2bbacaf11d67,98845,Kevin Swanson,2024-08-01 09:38:21,Cappuccino,2,4.0,Cash,Julia Dennis,Kevin Swanson,Jr. Barista,15,2.0
2,e7447dc1-6289-4e54-8b27-9b295b18d67f,82456,Diana Collins,2024-08-01 05:03:38,Latte,3,6.0,Card,Nicholas Hernandez,Diana Collins,Sr. Barista,15,2.0
3,fd99c34d-9895-4209-905c-b245a9e78257,99854,Rebecca Edwards,2024-08-01 01:47:53,Cappuccino,3,6.0,Card,Nicholas Hernandez,Rebecca Edwards,Sr. Barista,12,2.0
4,427b3d1d-9fb9-48d8-862f-2405b58e1167,82456,Diana Collins,2024-08-01 01:35:16,Latte,2,4.0,Card,Shane Ibarra,Diana Collins,Sr. Barista,15,2.0


## Data cleaning.

Now it is finally time to clean the data.

From what I can see, there are a few changes that I can make:
1. Shorten the transaction IDs to the first 8 string.
2. Drop 1 employee name column 'emp_name_y" and renaming "emp_name_x".
3. Separate the date and time then drop the column 'date_time'
4. Rearrange the columns.
5. Finishing touches.

In [56]:
df_cleaning = df_merged.copy() # Always best practice to create new copies of your dataframes.

In [57]:
df_cleaning['trn_id'] = df_cleaning['trn_id'].str[:8] # Shorten the transaction IDs to the first 8 string.
df_cleaning = df_cleaning.rename(columns={'emp_name_x' : 'emp_name'}) # Drop 1 employee name column 'emp_name_y" and renaming "emp_name_x".
df_cleaning = df_cleaning.drop(columns=['emp_name_y'])
df_cleaning['date'] = df_cleaning['date_time'].dt.date # Separate the date and time then drop the column 'date_time'
df_cleaning['time'] = df_cleaning['date_time'].dt.time
df_cleaning = df_cleaning.drop(columns=['date_time'])
df_cleaning.head()

,trn_id,emp_id,emp_name,coffee_type,purchase_qty,total_price,pay_method,customer_name,job_title,hr_salary,price,date,time
0,80dbda09,98845,Kevin Swanson,Latte,4,8.0,Card,Brandon Johnson,Jr. Barista,15,2.0,2024-08-01,10:17:03
1,a0ee1475,98845,Kevin Swanson,Cappuccino,2,4.0,Cash,Julia Dennis,Jr. Barista,15,2.0,2024-08-01,09:38:21
2,e7447dc1,82456,Diana Collins,Latte,3,6.0,Card,Nicholas Hernandez,Sr. Barista,15,2.0,2024-08-01,05:03:38
3,fd99c34d,99854,Rebecca Edwards,Cappuccino,3,6.0,Card,Nicholas Hernandez,Sr. Barista,12,2.0,2024-08-01,01:47:53
4,427b3d1d,82456,Diana Collins,Latte,2,4.0,Card,Shane Ibarra,Sr. Barista,15,2.0,2024-08-01,01:35:16


In [60]:
# Rearrange the columns.
reorder = [
     'trn_id', 'emp_id', 'hr_salary', 'job_title', 'emp_name',
     'date', 'time', 'coffee_type', 'purchase_qty', 'price',
     'total_price', 'pay_method', 'customer_name'
]

df_reorder = df_cleaning[reorder]
df_reorder.head()

,trn_id,emp_id,hr_salary,job_title,emp_name,date,time,coffee_type,purchase_qty,price,total_price,pay_method,customer_name
0,80dbda09,98845,15,Jr. Barista,Kevin Swanson,2024-08-01,10:17:03,Latte,4,2.0,8.0,Card,Brandon Johnson
1,a0ee1475,98845,15,Jr. Barista,Kevin Swanson,2024-08-01,09:38:21,Cappuccino,2,2.0,4.0,Cash,Julia Dennis
2,e7447dc1,82456,15,Sr. Barista,Diana Collins,2024-08-01,05:03:38,Latte,3,2.0,6.0,Card,Nicholas Hernandez
3,fd99c34d,99854,12,Sr. Barista,Rebecca Edwards,2024-08-01,01:47:53,Cappuccino,3,2.0,6.0,Card,Nicholas Hernandez
4,427b3d1d,82456,15,Sr. Barista,Diana Collins,2024-08-01,01:35:16,Latte,2,2.0,4.0,Card,Shane Ibarra


In [61]:
df_reorder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   trn_id         2494 non-null   object 
 1   emp_id         2494 non-null   int64  
 2   hr_salary      2494 non-null   int64  
 3   job_title      2494 non-null   object 
 4   emp_name       2494 non-null   object 
 5   date           2494 non-null   object 
 6   time           2494 non-null   object 
 7   coffee_type    2494 non-null   object 
 8   purchase_qty   2494 non-null   int64  
 9   price          2494 non-null   float64
 10  total_price    2494 non-null   float64
 11  pay_method     2494 non-null   object 
 12  customer_name  2494 non-null   object 
dtypes: float64(2), int64(3), object(8)
memory usage: 253.4+ KB


Upon inspection, I noticed that date is Dtype Objects which will be a problem if not fixed.

Let me fix that.

In [102]:
df_dt = df_reorder.copy() # Always best practice to create new copies of your dataframes.


In [108]:
df_dt['date'] = pd.to_datetime(df_dt['date'], format='%Y-%m-%d')
df_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   trn_id         2494 non-null   object        
 1   emp_id         2494 non-null   int64         
 2   hr_salary      2494 non-null   int64         
 3   job_title      2494 non-null   object        
 4   emp_name       2494 non-null   object        
 5   date           2494 non-null   datetime64[ns]
 6   time           2494 non-null   object        
 7   coffee_type    2494 non-null   object        
 8   purchase_qty   2494 non-null   int64         
 9   price          2494 non-null   float64       
 10  total_price    2494 non-null   float64       
 11  pay_method     2494 non-null   object        
 12  customer_name  2494 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(7)
memory usage: 253.4+ KB


In [113]:
df_dt.head()

,trn_id,emp_id,hr_salary,job_title,emp_name,date,time,coffee_type,purchase_qty,price,total_price,pay_method,customer_name
0,80dbda09,98845,15,Jr. Barista,Kevin Swanson,2024-08-01,10:17:03,Latte,4,2.0,8.0,Card,Brandon Johnson
1,a0ee1475,98845,15,Jr. Barista,Kevin Swanson,2024-08-01,09:38:21,Cappuccino,2,2.0,4.0,Cash,Julia Dennis
2,e7447dc1,82456,15,Sr. Barista,Diana Collins,2024-08-01,05:03:38,Latte,3,2.0,6.0,Card,Nicholas Hernandez
3,fd99c34d,99854,12,Sr. Barista,Rebecca Edwards,2024-08-01,01:47:53,Cappuccino,3,2.0,6.0,Card,Nicholas Hernandez
4,427b3d1d,82456,15,Sr. Barista,Diana Collins,2024-08-01,01:35:16,Latte,2,2.0,4.0,Card,Shane Ibarra


## Exporting to CSV

Now that the data is clean and ready for visualization.
I am going to export it into a csv file.

In [111]:
df_final = df_dt.copy()
df_final.to_csv('updated_coffee_sales_cleaned.csv', index=False)